# Shapley Additive Explanations (SHAP) of QRF Predictions

Version 21 December 2022, Selina Kiefer

### Input: csv-file, pt-file
continuous timeseries of input data (e.g. statistics of meteorological predictor fields), Quantile Random Forests model in pt-format
### Output: png-file
results of SHAP analysis plotted in png-format

#### Set the paths' to the defined functions, the style sheet for plotting and tthe configuration file and set its name

In [ ]:
# Set the path to the defined functions.
PATH_defined_functions = './Defined_Functions/'

In [ ]:
# Set the path and name of the style file which should be used for plotting.
style_file_for_plotting = './Style_File_Matplotlib.mplstyle'

In [ ]:
# Set the path and name of the configuration file.
PATH_configurations = './Configuration_Files/'
ifile_configurations = 'Configurations_QRF_SHAP.yaml'

#### Import the necessary python packages and functions
Nothing needs to be changed here.

In [ ]:
# Import the necessary python packages.
import yaml
import calendar
from collections import defaultdict
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import xarray as xr
import matplotlib.pyplot as plt
import torch
from skranger.ensemble import RangerForestRegressor

import shap
from shap import TreeExplainer
from skranger.utils.shap import shap_patch

In [ ]:
# Import the necessary python packages and functions.
import sys
sys.path.insert(1, PATH_defined_functions)
from read_in_csv_data import *

#### Read in the style sheet for plotting

In [ ]:
# Load the style sheet to be used by matplotlib for plotting. This will update the plotting
# parameters to e.g. have the right font, font size and figure size. The latter is adjusted to
# the textwidth of the LaTeX-document in order to avoid re-scaling the plot and changing 
# thereby the font size again.
plt.style.use(style_file_for_plotting)

#### Read in the configuration file and the data specified in it

In [ ]:
# Read in the configuration file (nothing needs to be changed here).
with open(PATH_configurations+ifile_configurations) as f:
    config = yaml.safe_load(f)

In [ ]:
# Read in the input data and remove any unnamed columns as well as the index column (nothing 
# needs to be changed here).
df_input_data = read_in_csv_data(config['PATH_input_data'], config['ifile_input_data'])
df_input_data = df_input_data.loc[:, ~df_input_data.columns.str.contains('^Unnamed')]
df_input_data = df_input_data.drop(['index'], axis =1 )

In [ ]:
# Set the name of the columns containing the time and the variables of the input data.
time_column_name_input_data = df_input_data.columns[0]
var_column_name_input_data = df_input_data.columns[1:]

In [ ]:
# Check that everything is selected correctly (nothing needs to be changed here).
print('Predictors used for training the ML model: ')
print(var_column_name_input_data)
print('Name of the column containing the time: ')
print(time_column_name_input_data)
print('Dataframe containing the predictors: ')
df_input_data.head()

#### Preparing the input data for the SHAP analysis
From here on, nothing needs to be changed.

In [ ]:
# A list containing the names of the input features is created. 
df_input_features = df_input_data.drop([time_column_name_input_data], axis=1)
input_features = df_input_features.columns.values

In [ ]:
# In order to extract the different time periods for the SHAP analysis, the index of the dataframe
# containing the input data is set to the time. The time column is converted beforehand into a
# datetime-object.
df_input_data[time_column_name_input_data] = pd.to_datetime(df_input_data[time_column_name_input_data])
df_input_data = df_input_data.set_index(time_column_name_input_data)

#### Defining the quantiles used for predicting by the QRF models

In [ ]:
# For predicting with the QRF-model, a list with the desired quantiles is created. 
if config['distributed_evenly']:
    list_quantiles_qrf = list(np.round(np.linspace(0, 1,config['number_of_quantiles']), decimals=2))
else:
    list_quantiles_qrf = config['list_quantiles_qrf']

#### Calculate the mean prediction of the training dataset

In [ ]:
# At first, the QRF model is loaded and the start as well as the end of the winter to be predicted set. Then, this
# winter is excluded from the input data forming the training dataset. In a next step, a probabilistic forecast on 
# the training dataset is done by QRF model. This forecast is then averaged over all quantiles and afterwards over
# all predicted days creating the mean prediction of the training dataset.
predictions = []
forecast_dates = []
winterly_values_all_winters = []

quantile_regresssion_forest = torch.load(config['PATH_model']+config['ifile_name_model'])

start_winter_to_be_excluded = datetime(config['start_year_of_winter'], config['start_month_winter'], config['start_day_winter'])

end_winter_to_be_excluded = datetime(config['start_year_of_winter']+1, config['end_month_winter'], config['end_day_winter'])

start_winter_to_be_excluded_minus_1_day = start_winter_to_be_excluded - timedelta(days=1)
end_winter_to_be_excluded_plus_1_day = end_winter_to_be_excluded + timedelta(days=1)

df_X_train = df_input_data.loc[(df_input_data.index < start_winter_to_be_excluded_minus_1_day) | (df_input_data.index > end_winter_to_be_excluded_plus_1_day)]    
df_X_train = df_X_train.reset_index()
df_X_train = df_X_train.drop([time_column_name_input_data], axis=1)
X_train = np.array(df_X_train)   

mean_prediction_value = np.mean(quantile_regresssion_forest.predict_quantiles(X_train, quantiles=list_quantiles_qrf), axis=1)
mean_prediction_value = np.mean(mean_prediction_value)

#### Calculation of the actual prediction of given time periods as well as calculating and plotting SHAP analysis of these time periods

In [ ]:
# Here, the SHAP analysis takes place. At first, the desired time period for the analysis is extracted from the input
# data. Then, the probabilistic prediction of the QRF model for this time period is obtained. This prediction is
# first averaged over all ensemble members and then over all predicted days creating the actual prediction of the QRF
# for the time period. As a next step, the SHAP analysis (with TreeExplainer) is done for every day in the time
#  period(.shap_values). Then, the mean and the standard deviation of these SHAP values are calculated and the top 5 
# positive and negative contributing predictors plotted as a bar plot together with the mean prediction 
# of the training dataset and the actual prediction of the QRF for the time period. The difference between the latter
# 2 is roughly the same as the sum over all SHAP values. It does not fit perfectly well since here averages over
# probabilistic forecasts are used instead of deterministic forecasts. This procedure is then done for every 
# specified time period in the configuration file separately. 
list_str_times = []
list_actual_prediction_value = []
list_temperature_periods = []

for i in range(len(config['start_dates_for_SHAP'])):
    
    predictions = []
    forecast_dates = []
    winterly_values_all_winters = []
    
    start_period = datetime.strptime(config['start_dates_for_SHAP'][i], '%Y-%m-%d')
    end_period = datetime.strptime(config['end_dates_for_SHAP'][i], '%Y-%m-%d')
    
    list_str_times.append(start_period.strftime('%d %b %Y')+' - '+end_period.strftime('%d %b %Y'))

    temperature_period = config['time_period_name'][i]
    list_temperature_periods.append(temperature_period)
    
    start_period_minus_1_day = start_period - timedelta(days=1)-timedelta(days=config['lead_time'])
    end_period_plus_1_day = end_period + timedelta(days=1)-timedelta(days=config['lead_time'])

    df_X_val = df_input_data.loc[(df_input_data.index > start_period_minus_1_day) & (df_input_data.index < end_period_plus_1_day)]    
    df_X_val = df_X_val.reset_index()
        
    X_val = df_X_val.drop([time_column_name_input_data], axis=1)

    actual_prediction_value = np.mean(quantile_regresssion_forest.predict_quantiles(X_val, quantiles=list_quantiles_qrf), axis=1)
    actual_prediction_value = np.mean(actual_prediction_value)  
    list_actual_prediction_value.append(actual_prediction_value)
    
    with shap_patch():
        explainer = TreeExplainer(model=quantile_regresssion_forest)
    
    daily_values = explainer.shap_values(X_val)
    time_period_values_mean = np.mean(daily_values, axis=0)
    time_period_values_std = np.std(daily_values, axis=0)
    
    df_shap_values = pd.DataFrame(time_period_values_mean)
    df_shap_values['Input Features'] = input_features
    df_shap_values['Standard Deviation'] = time_period_values_std

    df_shap_values_sorted = df_shap_values.sort_values(0, ascending=False)
    shap_mean_values_sorted = df_shap_values_sorted[0]
    shap_std_values_sorted = df_shap_values_sorted['Standard Deviation']
    input_features_sorted = df_shap_values_sorted['Input Features']

    #text_for_on_plot = 'Mean Prediction: ' + str(np.round(mean_prediction_value, decimals=2)) +' '+config['unit_of_ground_truth_and_prediction']+'\nActual Prediction: ' + str(np.round(actual_prediction_value, decimals=2)) +' '+config['unit_of_ground_truth_and_prediction']+'\nDifference: ' + str(np.round(np.round(actual_prediction_value, decimals=2)-np.round(mean_prediction_value, decimals=2), decimals=2)) +' '+config['unit_of_ground_truth_and_prediction']
    # don't plot the text on the plot for paper, rather create a table from the csv-file created later.
    
    fig,ax = plt.subplots()
    #plt.text(0,-0.35, text_for_on_plot, transform=ax.transAxes, verticalalignment='bottom')
    plt.barh(input_features_sorted[0:5], shap_mean_values_sorted[0:5], xerr=shap_std_values_sorted[0:5], label=input_features_sorted[0:5], color = 'r', alpha=0.6, ecolor='darkred', capsize=3)
    plt.barh(input_features_sorted[len(input_features)-5:len(input_features)], shap_mean_values_sorted[len(input_features)-5:len(input_features)], xerr=shap_std_values_sorted[len(input_features)-5:len(input_features)], label=input_features_sorted[len(input_features)-5:len(input_features)], color = 'b', alpha=0.6, ecolor='darkblue', capsize=3)
    plt.gca().invert_yaxis()
    plt.title(config['model_name']+', '+temperature_period+'\n '+start_period.strftime('%d %b %Y')+' - '+end_period.strftime('%d %b %Y'))
    plt.xlabel('SHAP Values')
    plt.savefig(config['PATH_plots']+'SHAP_'+config['model_name']+'_'+config['ground_truth']+'_'+config['location_ground_truth']+'_input_'+config['input_data']+'_'+config['location_input']+'_Lead_'+str(config['lead_time'])+'d_'+temperature_period.replace(' ', '_')+'_'+start_period.strftime('%d_%b_%Y')+'_'+end_period.strftime('%d_%b_%Y')+'.png', bbox_inches = 'tight')  
    #plt.show() # not used since the size of the jupyter notebook is really big
    plt.close() # only used since the size of the jupyter notebook is really big

In [ ]:
# Create a pandas dataframe with the mean and actual prediction as well as the difference.
df_details_shap_analysis = pd.DataFrame(list_str_times)
df_details_shap_analysis['Temperature Period'] = list_temperature_periods
df_details_shap_analysis['Mean Pred.'] = mean_prediction_value
df_details_shap_analysis['Actual Pred.'] = list_actual_prediction_value
df_details_shap_analysis['Difference'] = list_actual_prediction_value-mean_prediction_value
df_details_shap_analysis = df_details_shap_analysis.rename(columns={0 : 'time'})

In [ ]:
# Take only the analysis of the cold periods (this is done for a nice representation in the paper).
df_cold_periods = df_details_shap_analysis.where(df_details_shap_analysis['Temperature Period'] == 'Cold Period')
df_cold_periods = df_cold_periods.dropna()
df_cold_periods [' '] = list(map(chr, range(97, 97+(len(df_cold_periods))))) # create a list of lowercase letters
df_cold_periods = df_cold_periods.drop(['time', 'Temperature Period'], axis=1)
df_cold_periods = df_cold_periods.set_index(' ')
df_cold_periods

In [ ]:
# Save in csv-format.
df_cold_periods.to_csv(config['PATH_plots']+'SHAP_'+config['model_name']+'_'+config['ground_truth']+'_'+config['location_ground_truth']+'_input_'+config['input_data']+'_'+config['location_input']+'_Lead_'+str(config['lead_time'])+'d_cold_periods'+'.csv')

In [ ]:
# Repeat the same for the warm periods.
# Take only the analysis of the warm periods (this is done for a nice representation in the paper).
df_warm_periods = df_details_shap_analysis.where(df_details_shap_analysis['Temperature Period'] == 'Warm Period')
df_warm_periods = df_warm_periods.dropna()
df_warm_periods [' '] = list(map(chr, range(97, 97+(len(df_warm_periods))))) # create a list of lowercase letters
df_warm_periods = df_warm_periods.drop(['time', 'Temperature Period'], axis=1)
df_warm_periods = df_warm_periods.set_index(' ')
df_warm_periods

In [ ]:
# Save in csv-format.
df_cold_periods.to_csv(config['PATH_plots']+'SHAP_'+config['model_name']+'_'+config['ground_truth']+'_'+config['location_ground_truth']+'_input_'+config['input_data']+'_'+config['location_input']+'_Lead_'+str(config['lead_time'])+'d_warm_periods'+'.csv')

In [ ]:
# End of Program